# 2 - Cleanup (and maybe processing)
Now that I have merged the downloaded data, I can start studying it to solve the issues thtat can by solved by deleting things.

In [40]:
import pandas as pd


data = pd.read_csv('data\set_movies_sample_big.csv')

In [41]:
data

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,index.1,rating,genres.1,mpaa,description,votes,index.2,rt_critics_score,rt_audience_score
0,5252956,tt2258239,movie,8 First Dates,8 pervykh svidaniy,0,2012,\N,87,Comedy,0,6.1,['comedy'],Not Rated,"[""Vera is a TV star, Nikita is a veterinarian....",1051.0,0,NaN,NaN
1,282345,tt0294888,movie,Sakariba blues,Sakariba blues,0,1968,\N,108,"Action,Crime",1,NaN,"['action', 'crime']",NaN,NaN,NaN,1,NaN,NaN
2,5843848,tt3503004,movie,Polk Road: The Homecoming,Polk Road: The Homecoming,0,2012,\N,79,"Comedy,Mystery,Thriller",2,7.6,"['comedy', 'mystery', 'thriller']",Not Rated,"['After the events that occurred on Polk Road,...",7.0,2,NaN,NaN
3,5194598,tt2191618,movie,"Every Everything: The Music, Life & Times of G...","Every Everything: The Music, Life & Times of G...",0,2013,\N,97,"Documentary,Music",3,7.1,"['documentary', 'music']",Not Rated,"['In marked contrast to ""Color Me Obsessed,"" d...",105.0,3,NaN,NaN
4,29878,tt0030409,movie,Man from Music Mountain,Man from Music Mountain,0,1938,\N,58,Western,4,5.9,['western'],Passed,"[""When nasty land developers try to bilk hones...",153.0,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,341841,tt0356784,movie,The Highwayman,Katia tarik,0,1959,\N,\N,"Crime,Drama,Thriller",495,7.4,"['crime', 'drama', 'thriller']",NaN,NaN,10.0,495,NaN,NaN
496,7949849,tt8207890,movie,Out of Breath,Out of Breath,0,2018,\N,\N,Documentary,496,7.2,['documentary'],NaN,"[""Saving lives in the world's most secretive s...",24.0,496,NaN,NaN
497,91938,tt0094011,movie,Under the Sun of Satan,Sous le soleil de Satan,0,1987,\N,98,"Drama,Fantasy",497,6.7,"['drama', 'fantasy']",TV-MA,['A priest stuck in a rural congregation and b...,3338.0,497,NaN,NaN
498,184125,tt0191262,movie,Magie moderne,Magie moderne,0,1931,\N,85,Comedy,498,NaN,['comedy'],NaN,NaN,NaN,498,NaN,NaN


## Columns
According to the definition in https://www.imdb.com/interfaces/ movies don't have an 'End Year', as that value is reserved for series. As such, this column has no use for me.

In [42]:
# data.endYear.value_counts()


In [43]:
cols_to_drop = []
cols_to_drop.append('endYear')

The column ``titleType`` has only movie as values, because I already filtered anything else at the beggining.

In [44]:
cols_to_drop.append('titleType')

The ``index`` column is here as a small mistake when merging the sources and can be dropped without issue. It happens 3 times, but they all will drop at the same time.
The ``tconst`` column, even though it will serve no purpose at modeling because it is just the ID in IMDb, will be kept in case I need to expand information in the future.

In [45]:
cols_to_drop.append('index')

I have two columns of genders, but the one called ''genres.1'' has the same information but in a better format, so I will delete the other.

In [46]:
cols_to_drop.append('index')
cols_to_drop.append('index.1')
cols_to_drop.append('index.2')

There are also two columns for names and they won't affect much, but still I will keep the names in English from ''primaryTitle''.

In [47]:
cols_to_drop.append('originalTitle')

Now it's time to drop some columns.

In [48]:
data = data.drop(cols_to_drop, 1)
data.head(1)

C:\Users\carlo\AppData\Local\Temp/ipykernel_17316/1662013735.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(cols_to_drop, 1)


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,rating,genres.1,mpaa,description,votes,rt_critics_score,rt_audience_score
0,tt2258239,8 First Dates,0,2012,87,Comedy,6.1,['comedy'],Not Rated,"[""Vera is a TV star, Nikita is a veterinarian....",1051.0,NaN,NaN


## NaNs

In [49]:
data.isna().sum()/len(data)

tconst               0.000
primaryTitle         0.000
isAdult              0.000
startYear            0.000
runtimeMinutes       0.000
genres               0.000
rating               0.426
genres.1             0.000
mpaa                 0.792
description          0.306
votes                0.426
rt_critics_score     1.000
rt_audience_score    1.000
dtype: float64

From this overview I can get some conclussions:  
- The first columns are all populated because the come from an official database  
- the last 2 columns appear as filled with errors due to a problem with the scraper, I will try to solve it later.

### ``rating``
As I am working with a sample, just removing rows does not seem like a good option for now.

In [53]:
data.rating.describe()

count    287.000000
mean       6.142509
std        1.329285
min        2.100000
25%        5.500000
50%        6.300000
75%        7.100000
max        9.000000
Name: rating, dtype: float64